# Prototype debugger

In [52]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py
import datetime
import json
import math

import schedule
import time
from datetime import datetime, timedelta
from joblib import dump, load

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)

In [53]:
def get_margin_asset(name:str):
    """returns a dictionary with:
    - asset name
    - free
    - locked
    - borrowed
    - interest
    - netAsset"""
    return list(filter(lambda x: x['asset'] == name, client.get_margin_account()["userAssets"]))[0]

In [28]:
def get_isolated_margin_account(base_asset: str):
    """Returns dict for isolated margin account for base_asset. Enter base_asset as 'FET'. Do NOT include USDT"""
    c = client.get_isolated_margin_account()
    return list(filter(lambda x: x["baseAsset"]["asset"] == base_asset, c["assets"]))[0]

In [39]:
a = get_isolated_margin_account("FET")

In [51]:
a['marginLevel']

'999'

In [60]:
def get_filtered_dataframe(df):
    """filters columns and converts columsn to floats and ints respectively"""
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df = df.astype(np.float64)
    df["timestamp"] = df.timestamp.astype(np.int64)
    return df

def get_minutely_data(symbol:str, days=0.5):
    """smart gets minutely data. Enter symbol with USDT"""
    data_past = pd.read_csv(f"../data/{symbol}-past.csv")

    d = datetime.today() - timedelta(days=days)
    start_date = d.strftime("%d %b %Y %H:%M:%S")
    today = datetime.today().strftime("%d %b %Y %H:%M:%S")

    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, start_date, today, 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data = get_filtered_dataframe(data)

    index = data_past.index[(data_past['timestamp'] == data.iloc[0].timestamp)].tolist()[0]
    data = pd.concat([data_past[:index], data], ignore_index=True, sort=False)

    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
    data_latest = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data_latest = get_filtered_dataframe(data_latest)

    index = data.index[(data['timestamp'] == data_latest.iloc[0].timestamp)].tolist()[0]
    result = pd.concat([data[:index], data_latest], ignore_index=True, sort=False)
    result.to_csv(f"../data/{symbol}-past.csv", index=False)
    return result

def get_z_score(): 
    '''gets the latest z-score, given hedge ratio hr.
    Warning, sometimes it gives nan, just rerun (binance's fault)'''
    a = get_minutely_data(ASSET_A + BASE)
    b = get_minutely_data(ASSET_B + BASE)
    a.set_index("timestamp", inplace=True)
    b.set_index("timestamp", inplace=True)
    
    df = pd.to_numeric(a.open.rename("A")).to_frame()
    df["B"] = pd.to_numeric(b.open)
    
    df.dropna(inplace=True)
    
    results = sm.ols(formula="B ~ A", data=df[['B', 'A']]).fit()
    hr = results.params[1]
    spread = pd.Series((df['B'] - hr * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(2000).mean()
    spread["std"] =  spread.spread.rolling(2000).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    
    return spread.iloc[-1].zscore

In [61]:
ASSET_A = "CELR"
ASSET_B = "FET"
BASE = "USDT"
get_z_score()

2.06090901911535